In [2]:
import numpy as np
import pyquil.quil as pq
from pyquil.gates import H, X, Z, RZ, STANDARD_GATES

from grove.utils.utility_programs import ControlledProgramBuilder

STANDARD_GATE_NAMES = list(STANDARD_GATES.keys())
X_GATE = np.array([[0, 1], [1, 0]])
X_GATE_LABEL = "NOT"
HADAMARD_DIFFUSION_LABEL = "HADAMARD_DIFFUSION"


def diffusion_program(qubits):
    diffusion_program = pq.Program()
    dim = 2 ** len(qubits)
    hadamard_diffusion_matrix = np.diag([1.0] + [-1.0] * (dim - 1))
    diffusion_program.defgate(HADAMARD_DIFFUSION_LABEL, hadamard_diffusion_matrix)
    instruction_tuple = (HADAMARD_DIFFUSION_LABEL,) + tuple(qubits)
    diffusion_program.inst(instruction_tuple)
    return diffusion_program


def amplification_circuit(algorithm, oracle, qubits, num_iter, decompose_diffusion=False):
    """
    Returns a program that does ``num_iter`` rounds of amplification, given a measurement-less
    algorithm, an oracle, and a list of qubits to operate on.

    :param Program algorithm: A program representing a measurement-less algorithm run on qubits.
    :param Program oracle: An oracle maps any basis vector ``|psi>`` to either ``+|psi>`` or
        ``-|psi>`` depending on whether ``|psi>`` is in the desirable subspace or the undesirable
        subspace.
    :param Sequence qubits: the qubits to operate on
    :param int num_iter: number of iterations of amplifications to run
    :param bool decompose_diffusion: If True, decompose the Grover diffusion gate into two qubit
     gates. If False, use a defgate to define the gate.
    :return: The amplified algorithm.
    :rtype: Program
    """
    program = pq.Program()

    uniform_superimposer = pq.Program().inst([H(qubit) for qubit in qubits])
    program += uniform_superimposer
    if decompose_diffusion:
        diffusion = decomposed_diffusion_program(qubits)
    else:
        diffusion = diffusion_program(qubits)
    # To avoid redefining gates, we collect them before building our program.
    defined_gates = oracle.defined_gates + algorithm.defined_gates + diffusion.defined_gates
    for _ in range(num_iter):
        program += (oracle.instructions
                 + algorithm.dagger().instructions
                 + diffusion.instructions
                 + algorithm.instructions)
    # We redefine the gates in the new program.
    for gate in defined_gates:
        program.defgate(gate.name, gate.matrix)
    return program


def decomposed_diffusion_program(qubits):
    """
    Constructs the diffusion operator used in Grover's Algorithm, acted on both sides by an
    a Hadamard gate on each qubit. Note that this means that the matrix representation of this
    operator is diag(1, -1, ..., -1). In particular, this decomposes the diffusion operator, which
    is a :math:`2**{len(qubits)}\times2**{len(qubits)}` sparse matrix, into
     :math:`\mathcal{O}(len(qubits)**2) single and two qubit gates.

    See C. Lavor, L.R.U. Manssur, and R. Portugal (2003) `Grover's Algorithm: Quantum Database
    Search`_ for more information.

    .. _`Grover's Algorithm: Quantum Database Search`: https://arxiv.org/abs/quant-ph/0301079

    :param qubits: A list of ints corresponding to the qubits to operate on.
                   The operator operates on bistrings of the form
                   ``|qubits[0], ..., qubits[-1]>``.
    """
    program = pq.Program()
    if len(qubits) == 1:
        program.inst(Z(qubits[0]))
    else:
        program.inst([X(q) for q in qubits])
        program.inst(H(qubits[-1]))
        program.inst(RZ(-np.pi, qubits[0]))
        program += (ControlledProgramBuilder()
                              .with_controls(qubits[:-1])
                              .with_target(qubits[-1])
                              .with_operation(X_GATE)
                              .with_gate_name(X_GATE_LABEL).build())
        program.inst(RZ(-np.pi, qubits[0]))
        program.inst(H(qubits[-1]))
        program.inst([X(q) for q in qubits])
    return program

In [3]:
import numpy as np
import pyquil.quil as pq
from pyquil.gates import H

class Grover(object):
    """This class contains an implementation of Grover's algorithm using pyQuil. See `these notes`_
     by Dave Bacon for more information.

    .. _these notes: https://courses.cs.washington.edu/courses/cse599d/06wi/lecturenotes12.pdf
    """
    def __init__(self):
        self.unitary_function_mapping = None
        self.size = None
        self.qubits = None
        self.grover_circuit = None
        self.bit_map = None

    @staticmethod
    def _compute_grover_oracle_matrix(bitstring_map):
        """Computes the unitary matrix that encodes the oracle function for Grover's algorithm

        :param bitstring_map: dict with string keys corresponding to bitstrings,
         and integer values corresponding to the desired phase on the output state.
        :type bitstring_map: Dict[String, Int]
        :return: a numpy array corresponding to the unitary matrix for oracle for the given
         bitstring_map
        :rtype: numpy.ndarray
        """
        n_bits = len(list(bitstring_map.keys())[0])
        oracle_matrix = np.zeros(shape=(2 ** n_bits, 2 ** n_bits))
        for b in range(2 ** n_bits):
            pad_str = np.binary_repr(b, n_bits)
            phase_factor = bitstring_map[pad_str]
            oracle_matrix[b, b] = phase_factor
        return oracle_matrix

In [15]:
class Grover_Full(Grover):
    def __init__(self):
        super().__init__()
        
    def _construct_grover_circuit(self, num_iter=None):
        """Contructs an instance of Grover's Algorithm, using initialized values.

        :return: None
        :rtype: NoneType
        """
        oracle = pq.Program()
        oracle_name = "GROVER_ORACLE"
        oracle.defgate(oracle_name, self.unitary_function_mapping)
        oracle.inst(tuple([oracle_name] + self.qubits))
        self.grover_circuit = self.oracle_grover(oracle, self.qubits, num_iter)
        
    def _init_attr(self, bitstring_map, num_iter=None):
        """Initializes an instance of Grover's Algorithm given a bitstring_map.

        :param bitstring_map: dict with string keys corresponding to bitstrings, and integer
         values corresponding to the desired phase on the output state.
        :type bitstring_map: Dict[String, Int]
        :return: None
        :rtype: NoneType
        """
        self.bit_map = bitstring_map
        self.unitary_function_mapping = self._compute_grover_oracle_matrix(bitstring_map)
        self.size = self.unitary_function_mapping.shape[0]
        self.qubits = list(range(int(np.log2(self.size))))
        self._construct_grover_circuit(num_iter)

    def find_bitstring(self, cxn, bitstring_map, num_iter=None):
        """
        Runs Grover's Algorithm to find the bitstring that is designated by ``bistring_map``.

        In particular, this will prepare an initial state in the uniform superposition over all bit-
        strings, an then use Grover's Algorithm to pick out the desired bitstring.

        :param QVMConnection cxn: the connection to the Rigetti cloud to run pyQuil programs.
        :param bitstring_map: a mapping from bitstrings to the phases that the oracle should impart
            on them. If the oracle should "look" for a bitstring, it should have a ``-1``, otherwise
            it should have a ``1``.
        :type bitstring_map: Dict[String, Int]
        :return: Returns the bitstring resulting from measurement after Grover's Algorithm.
        :rtype: str
        """

        self._init_attr(bitstring_map, num_iter=num_iter)
        sampled_bitstring = cxn.run_and_measure(self.grover_circuit, self.qubits)[0]
        return "".join([str(bit) for bit in sampled_bitstring])
        
    @staticmethod
    def oracle_grover(oracle, qubits, num_iter=None):
        """Implementation of Grover's Algorithm for a given oracle.

        :param Program oracle: An oracle defined as a Program. It should send :math:`\ket{x}`
            to :math:`(-1)^{f(x)}\ket{x}`, where the range of f is {0, 1}.
        :param qubits: List of qubits for Grover's Algorithm.
        :type qubits: list[int or Qubit]
        :param int num_iter: The number of iterations to repeat the algorithm for.
                             The default is the integer closest to :math:`\frac{\pi}{4}\sqrt{N}`,
                             where :math:`N` is the size of the domain.
        :return: A program corresponding to the desired instance of Grover's Algorithm.
        :rtype: Program
        """
        if num_iter is None:
            num_iter = int(round(np.pi * 2 ** (len(qubits) / 2.0 - 2.0)))
        uniform_superimposer = pq.Program().inst([H(qubit) for qubit in qubits])
        amp_prog = amplification_circuit(uniform_superimposer, oracle, qubits, num_iter)
        return amp_prog

In [41]:
# Construct Partial Diffusion Gate
PARTIAL_HADAMARD_DIFFUSION_LABEL = "PARTIAL_HADAMARD_DIFFUSION"

def partial_diffusion_program(num_target_qubits, qubits):
    partial_diffusion_program = pq.Program()
    block_size = 2 ** num_target_qubits
    num_blocks = 2 ** (len(qubits) - num_target_qubits)
    
    partial_hadamard_diffusion_matrix = np.diag(([1.0] + [-1.0] * (block_size - 1)) * num_blocks)
    partial_diffusion_program.defgate(PARTIAL_HADAMARD_DIFFUSION_LABEL, partial_hadamard_diffusion_matrix)
    instruction_tuple = (PARTIAL_HADAMARD_DIFFUSION_LABEL,) + tuple(qubits)
    partial_diffusion_program.inst(instruction_tuple)
    
    return partial_diffusion_program

def decomposed_partial_diffusion_program(num_target_qubits, qubits):
    # TODO
    pass

def block_amplification_circuit(algorithm, oracle, qubits, num_block_iter, num_target_qubits, decompose_diffusion=False):
    program = pq.Program()
    
    if decompose_diffusion:
        partial_diffusion = decomposed_partial_diffusion_program(num_target_qubits, qubits)
    else:
        partial_diffusion = partial_diffusion_program(num_target_qubits, qubits)

    # To avoid redefining gates, we collect them before building our program.
    defined_gates = oracle.defined_gates + algorithm.defined_gates + partial_diffusion.defined_gates
        
    for _ in range(num_block_iter):
        program += (oracle.instructions
                 + algorithm.dagger().instructions
                 + partial_diffusion.instructions
                 + algorithm.instructions)
    # We redefine the gates in the new program.
    for gate in defined_gates:
        program.defgate(gate.name, gate.matrix)
    return program

In [38]:
class Grover_Partial(Grover):
    def __init__(self):
        super().__init__()

    def _construct_grover_circuit(self, num_block_iter=None, num_global_iter=None, num_target_qubits=None):
        oracle = pq.Program()
        oracle_name = "GROVER_ORACLE"
        oracle.defgate(oracle_name, self.unitary_function_mapping)
        oracle.inst(tuple([oracle_name] + self.qubits))
        self.grover_circuit = self.oracle_grover(oracle, self.qubits, num_block_iter=num_block_iter, num_global_iter=num_global_iter, num_target_qubits=num_target_qubits)

    def _init_attr(self, bitstring_map, num_block_iter=None, num_global_iter=None, num_target_qubits=None):
        """Initializes an instance of Grover's Algorithm given a bitstring_map.

        :param bitstring_map: dict with string keys corresponding to bitstrings, and integer
         values corresponding to the desired phase on the output state.
        :type bitstring_map: Dict[String, Int]
        :return: None
        :rtype: NoneType
        """
        self.bit_map = bitstring_map
        self.unitary_function_mapping = self._compute_grover_oracle_matrix(bitstring_map)
        self.size = self.unitary_function_mapping.shape[0]
        self.qubits = list(range(int(np.log2(self.size))))
        self._construct_grover_circuit(num_block_iter=num_block_iter, num_global_iter=num_global_iter, num_target_qubits=num_target_qubits)
        
    def find_bitstring(self, cxn, bitstring_map, num_block_iter=None, num_global_iter=None, num_target_qubits=None):
        self._init_attr(bitstring_map, num_block_iter=num_block_iter, num_global_iter=num_global_iter, num_target_qubits=num_target_qubits)
        sampled_bitstring = cxn.run_and_measure(self.grover_circuit, self.qubits)[0]
        return "".join([str(bit) for bit in sampled_bitstring])
    
    @staticmethod
    def oracle_grover(oracle, qubits, num_block_iter=None, num_global_iter=None, num_target_qubits=None):
        uniform_superimposer = pq.Program().inst([H(qubit) for qubit in qubits])
        
        if num_target_qubits is None:
            num_target_qubits = len(qubits)
        
        if num_global_iter is None:
            num_global_iter = int(round(np.pi * 2 ** (len(qubits) / 2.0 - 2.0)))
        global_amp_prog = amplification_circuit(uniform_superimposer, oracle, qubits, num_global_iter)
        
        if num_block_iter is None:
            num_block_iter = 0 # TODO
        block_amp_prog = block_amplification_circuit(uniform_superimposer, oracle, qubits, num_block_iter, num_target_qubits)
        
        return global_amp_prog + block_amp_prog

In [5]:
from pyquil.api import QVMConnection
from itertools import product
from mock import patch, Mock

In [16]:
def grover_specs(target_bitstring, cxn, num_iter=None):
    bit = ("0", "1")
    bitstring_map = {}
    target_bitstring_phase = -1
    nontarget_bitstring_phase = 1

    for bitstring in product(bit, repeat=len(target_bitstring)):
        bitstring = "".join(bitstring)
        if bitstring == target_bitstring:
            bitstring_map[bitstring] = target_bitstring_phase
        else:
            bitstring_map[bitstring] = nontarget_bitstring_phase
    
    grover = Grover_Full()
    grover._init_attr(bitstring_map, num_iter=num_iter)
    
    target_index = int(target_bitstring, 2)
    print(cxn.wavefunction(grover.grover_circuit))
    target_amplitude = cxn.wavefunction(grover.grover_circuit).amplitudes[::-1][target_index]
    print("Success Probability:", target_amplitude * target_amplitude.conjugate())
    
    found_bitstring = grover.find_bitstring(cxn, bitstring_map)
    print("Found Bitstring:", found_bitstring)
    assert "".join(found_bitstring) == target_bitstring, "Found bitstring is not the expected bitstring"

In [17]:
grover_specs(target_bitstring='0101', cxn=QVMConnection())

(-0.05078125+0j)|0000> + (-0.05078125+0j)|0001> + (-0.05078125+0j)|0010> + (-0.05078125+0j)|0011> + (-0.05078125+0j)|0100> + (-0.05078125+0j)|0101> + (-0.05078125+0j)|0110> + (-0.05078125+0j)|0111> + (-0.05078125+0j)|1000> + (-0.05078125+0j)|1001> + (0.98046875+0j)|1010> + (-0.05078125+0j)|1011> + (-0.05078125+0j)|1100> + (-0.05078125+0j)|1101> + (-0.05078125+0j)|1110> + (-0.05078125+0j)|1111>
Success Probability: (0.9613189697265582+0j)
Found Bitstring: 0101


In [39]:
def partial_grover_specs(target_bitstring, cxn, num_global_iter=None, num_block_iter=None, num_target_qubits=None):
    bit = ("0", "1")
    bitstring_map = {}
    target_bitstring_phase = -1
    nontarget_bitstring_phase = 1

    for bitstring in product(bit, repeat=len(target_bitstring)):
        bitstring = "".join(bitstring)
        if bitstring == target_bitstring:
            bitstring_map[bitstring] = target_bitstring_phase
        else:
            bitstring_map[bitstring] = nontarget_bitstring_phase
    
    grover = Grover_Partial()
    grover._init_attr(bitstring_map, num_block_iter=num_block_iter, num_global_iter=num_global_iter, num_target_qubits=num_target_qubits)
    
    target_index = int(target_bitstring, 2)
    print(cxn.wavefunction(grover.grover_circuit))
    target_amplitude = cxn.wavefunction(grover.grover_circuit).amplitudes[::-1][target_index]
    print("Success Probability:", target_amplitude * target_amplitude.conjugate())
    
    found_bitstring = grover.find_bitstring(cxn, bitstring_map)
    print("Found Bitstring:", found_bitstring)
    assert "".join(found_bitstring) == target_bitstring, "Found bitstring is not the expected bitstring"

In [42]:
partial_grover_specs(target_bitstring='0101', cxn=QVMConnection())

/Users/Family/Desktop/QuantumWorld/pyquil/pyquil/quil.py:132: UserWarning: Gate GROVER_ORACLE has already been defined in this program
  warnings.warn("Gate {} has already been defined in this program".format(instruction.name))


(-0.05078125+0j)|0000> + (-0.05078125+0j)|0001> + (-0.05078125+0j)|0010> + (-0.05078125+0j)|0011> + (-0.05078125+0j)|0100> + (-0.05078125+0j)|0101> + (-0.05078125+0j)|0110> + (-0.05078125+0j)|0111> + (-0.05078125+0j)|1000> + (-0.05078125+0j)|1001> + (0.98046875+0j)|1010> + (-0.05078125+0j)|1011> + (-0.05078125+0j)|1100> + (-0.05078125+0j)|1101> + (-0.05078125+0j)|1110> + (-0.05078125+0j)|1111>
Success Probability: (0.9613189697265582+0j)
Found Bitstring: 0101
